# Accelerated variational algorithms for digital quantum simulation of the many-body ground states

## 项目介绍

论文的目标是在数字量子模拟器（Digital Quantum Simulator, DQS）上制备一维链海森堡模型的基态（开放边界条件）$$H=J \sum_{i=1}^{N-1} \sigma^{i} \cdot \sigma^{i+1}$$
其中$J$为exchange coupling，是大于0的实数，$\sigma^{i}=\left(\sigma_{x}^{i}, \sigma_{y}^{i}, \sigma_{z}^{i}\right)$是第$i$个qubit上的Pauli算符组成的向量。

### 一、绝热演化
针对此目标，论文首先提出了使用Suzuki-Trotter expansion来模拟绝热演化的方案，即进行如下的绝热演化$$H_{a d}(t)=H_{o d d}+\frac{t}{T_{\max }} H_{e v e n}$$
其中$H_{o d d}=J \sum_{\text {odd } i} \sigma^{i} \cdot \sigma^{i+1}$，$H_{\text {even }}=J \sum_{\text {even } i} \sigma^{i} \cdot \sigma^{i+1}$，$0 \leq t \leq T_{\max }$，$H_{o d d}$的基态是容易制备的$$|\Psi(0)\rangle=\left|\psi^{-}\right\rangle \otimes \cdots \otimes\left|\psi^{-}\right\rangle$$其中$\left|\psi^{-}\right\rangle=(|01\rangle-|10\rangle) / \sqrt{2}$

当取$T_{\max } \sim N^{2}$时，就能满足绝热定理的条件，并且$t=T_{\max }$时$H_{a d}\left(T_{\max }\right)=H$

实现绝热演化的线路图见论文Fig.1

### 二、VQE
实现绝热演化的线路深度过高，考虑使用VQE，以在保障精度的同时降低线路深度。绝热演化的严格性由物理定律保证，因此在设计VQE所使用的参数化量子线路（Parameterized Quantum Circuit, PQC）时，论文借鉴了绝热演化的线路结构。VQE的线路图见论文Fig.3

在此基础上，论文对如何缩短VQE参数更新的经典优化循环次数作了进一步的讨论，提出了3种策略。
#### 1. 随机初始化
#### 2. 比特数递归
#### 3. 层数递归
策略的具体内容在复现过程中详细介绍。

### 三、其他讨论
论文最后讨论了以下四点：
#### 1. 线路的扩展性
相同的线路可以扩展到各向异性的海森堡模型，只需要门参数做一些微调，不用引入新的参数。
#### 2. 镜面对称性的作用
文中使用的PQC线路的结构中，每一层的PQC最后都有一层所有qubit上的相位门。由于论文中考虑的哈密顿量具有镜面对称性，在前文的讨论中，这一层相位门的参数也被强加了这一对称性。但作者发现，倘若取消相位门的镜面对称性，VQE反而能以更快的速度收敛。
#### 3. VQE线路设计思路
作者提出，VQE线路借鉴绝热演化的Suzuki-Trotter expansion这一思路是普适的。
#### 4. 退相干的影响
将线路固定为某一次VQE的最终参数，引入双量子比特门的随机的噪声扰动或失相，考察对保真度的影响。

## 复现过程

首先，安装、导入依赖包。

In [3]:
import numpy as np
from mindquantum import Circuit, X, H, RZ, RY, PhaseShift, Hamiltonian, Simulator, QubitOperator, Hamiltonian, MQAnsatzOnlyLayer
import mindspore as ms
from mindspore.common.parameter import Parameter
import mindspore.context as context

context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")

参考Fig.1a，自定义一个单参数双量子比特门N（论文中的$\mathcal{N}(\theta)$），这个门使用参数f'p{count}'，作用于 i 和 i + 1 号量子比特（也即第$i+1$和第$i+2$个量子比特）。

In [4]:
def N(count, i):
    temp = Circuit()
    temp += RZ(np.pi / 2).on(i + 1)
    temp += X.on(i, i + 1)
    temp += RZ({f'p{count}' : 2}).on(i)
    temp += RZ(-np.pi / 2).on(i)
    temp += RY(np.pi / 2).on(i + 1)
    temp += RY({f'p{count}' : -2}).on(i + 1)
    temp += X.on(i + 1, i)
    temp += RY({f'p{count}' : 2}).on(i + 1)
    temp += RY(-np.pi / 2).on(i + 1)
    temp += X.on(i, i + 1)
    temp += RZ(-np.pi / 2).on(i)
    return temp

### Strategy 1: random initialization.

定义模型的量子比特数num（论文中的$N$），和VQE线路的层数m（论文中的$M_{VQE}$）。根据论文Table.1，$N$取$4,8,10$时，若要达到$F=0.99$保真度，$M_{VQE}$应分别取$2,3,3$. 由于本文考虑哈密顿量的镜面对称性，因此要求$N$必须是偶数。

In [5]:
NMtable = {4:2, 8:3, 10:3, 16:5, 20:6}
num = 4
m = NMtable[num]
assert num % 2 == 0
print(num, m)

4 2


制备初态$|\Psi(0)\rangle=\left|\psi^{-}\right\rangle \otimes \cdots \otimes\left|\psi^{-}\right\rangle$，其中$\left|\psi^{-}\right\rangle=(|01\rangle-|10\rangle) / \sqrt{2}$，这个量子态是$H_{o d d}=J \sum_{\operatorname{odd} i} \sigma^{i} \cdot \sigma^{i+1}$的基态，如前文所述，是绝热演化的起点，因此也是参数化量子线路的起点。

In [7]:
encoder = Circuit()
for i in range(0, num, 2):
    encoder += X.on(i)
    encoder += X.on(i + 1)
    encoder += H.on(i)
    encoder += X.on(i + 1, i)
    
encoder.summary()
encoder

=======Circuit Summary=======
|Total number of gates  : 8.|
|Parameter gates        : 0.|
|with 0 parameters are  : . |
|Number qubit of circuit: 4 |


q0: ──X────H────●──
                │
q1: ──X─────────X──

q2: ──X────H────●──
                │
q3: ──X─────────X──

参考Fig.3a构建ansatz线路，由于encoder中不含有参数，最后直接将其与ansatz合并。

In [10]:
ansatz = Circuit()
count = 1

for j in range(m):
    for i in range(0, num - 1, 2):
        ansatz += N(count, i)
        count += 1
    for i in range(1, num - 1, 2):
        ansatz += N(count, i)
        count += 1
    for i in range(0, num // 2):
        ansatz += PhaseShift('p%d' % count).on(i)
        ansatz += PhaseShift({'p%d' % count : -1}).on(num - i - 1)
        count += 1

ansatz = encoder + ansatz
ansatz.summary()
ansatz

==========================Circuit Summary==========================
|Total number of gates  : 82.                                     |
|Parameter gates        : 26.                                     |
|with 10 parameters are : p1, p2, p3, p4, p5, p6, p7, p8, p9, p10.|
|Number qubit of circuit: 4                                       |


q0: ──X────H────●───────────────X────RZ(2*p1)─────RZ(-π/2)────●────────────────────────────X────RZ(-π/2)─────PS(p4)──────────────────────────────────────────────────────────────────────────────────────────────────────────X────────RZ(2*p6)─────RZ(-π/2)───────●───────────────────────────────────X───────RZ(-π/2)─────PS(p9)─────────────────────────────────────────────────────────────────────────
                │               │                             │                            │                                                                                                                                 │                                    │                                   │
q1: ──X─────────X────RZ(π/2)────●────RY(π/2)─────RY(-2*p1)────X────RY(2*p1)────RY(-π/2)────●───────────────────────────X────RZ(2*p3)─────RZ(-π/2)────●────────────────────────────X────RZ(-π/2)────PS(p5)────RZ(π/2)─────────●────────RY(π/2)─────RY(-2*p6)───────X────────RY(2*p6)────RY(-π/2)───────●──────────X────────RZ(2*p8)─────RZ(-π/2)────●────────────────────────────X────RZ(-π/2)────PS(p10)──
                                                                                                                       │                             │                            │                                                                                                                              │                                 │                            │
q2: ──X────H────●───────────────X────RZ(2*p2)─────RZ(-π/2)────●────────────────────────────X────RZ(-π/2)────RZ(π/2)────●────RY(π/2)─────RY(-2*p3)────X────RY(2*p3)────RY(-π/2)────●────PS(-p5)───────X───────RZ(2*p7)─────RZ(-π/2)───────●────────────────────────────────────X────────RZ(-π/2)────RZ(π/2)───────●────────RY(π/2)─────RY(-2*p8)────X────RY(2*p8)────RY(-π/2)────●────PS(-p10)─────────────
                │               │                             │                            │                                                                                                         │                                   │                                    │
q3: ──X─────────X────RZ(π/2)────●────RY(π/2)─────RY(-2*p2)────X────RY(2*p2)────RY(-π/2)────●────PS(-p4)─────RZ(π/2)──────────────────────────────────────────────────────────────────────────────────●───────RY(π/2)─────RY(-2*p7)───────X─────────RY(2*p7)────RY(-π/2)───────●────────PS(-p9)────────────────────────────────────────────────────────────────────────────────────────────────────────────

参考式(1)，构建哈密顿量$H=J \sum_{i=1}^{N-1} \sigma^{i} \cdot \sigma^{i+1}$，这里定义$J=1$

In [11]:
J = 1
ham = QubitOperator('')
for i in range(num - 1):
    ham += QubitOperator('X%d X%d' % (i, i + 1), J)
    ham += QubitOperator('Y%d Y%d' % (i, i + 1), J)
    ham += QubitOperator('Z%d Z%d' % (i, i + 1), J)
ham = Hamiltonian(ham - QubitOperator(''))

print(ham)

1 [X0 X1] +
1 [Y0 Y1] +
1 [Z0 Z1] +
1 [X1 X2] +
1 [Y1 Y2] +
1 [Z1 Z2] +
1 [X2 X3] +
1 [Y2 Y3] +
1 [Z2 Z3] 


构建参数化量子线路，对参数进行随机初始化（标准正态分布）。

In [9]:
val = np.random.randn(count - 1)

sim = Simulator('mqvector', num)
pqc = sim.get_expectation_with_grad(ham, ansatz)
pqcnet = MQAnsatzOnlyLayer(pqc)
pqcnet.weight = Parameter(ms.Tensor(val, pqcnet.weight.dtype))

initial_energy = pqcnet()
print("Initial energy: %20.16f" % (initial_energy.asnumpy()))

Initial energy:  -0.0997337326407433


使用Adam算法进行梯度优化，各参数取值如论文所述。当相邻两步的能量差小于阈值时（这里设定为1e-8），判定为收敛，更新结束。

In [10]:
optimizer = ms.nn.Adam(pqcnet.trainable_params(), learning_rate = 0.01, beta1 = 0.9, beta2 = 0.999, eps = 1e-8)
train_pqcnet = ms.nn.TrainOneStepCell(pqcnet, optimizer)

eps = 1.e-8
print("eps: ", eps)
energy_diff = eps * 1000
energy_last = initial_energy.asnumpy() + energy_diff
iter_idx = 0
while (abs(energy_diff) > eps):
    energy_i = train_pqcnet().asnumpy()
    if iter_idx % 100 == 0:
        print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
    energy_diff = energy_last - energy_i
    energy_last = energy_i
    iter_idx += 1

print("Optimization completed at step %3d" % (iter_idx - 1))
print("Optimized energy: %20.16f" % (energy_i))
print("Optimized amplitudes: \n", pqcnet.weight.asnumpy())

eps:  1e-08
Step   0 energy  -0.0997337326407433
Step 100 energy  -5.9991140365600586
Step 200 energy  -6.3746271133422852
Optimization completed at step 285
Optimized energy:  -6.4641013145446777
Optimized amplitudes: 
 [ 0.4028975   1.7767195   0.78539985 -0.44342345  0.44359204 -0.23663777
  0.5444191  -0.54652977 -1.57108    -1.5708411 ]


使用accvqeexact.py求解哈密顿量严格对角化的结果，该程序输出一个实数表示基态能量，可以与VQE的结果进行对比。\
N取4,8,10时，基态能量分别为-6.464101615137754, -13.499730394751591, -17.032140829131546.

Strategy 1完整源代码见src/random initialization.py

### Strategy 2: qubit recursive.

这个策略先在较小的系统（目标尺寸的一半，可以递归至最小的大于等于4的偶数）上进行VQE，将此步得到的最终参数作为目标系统的VQE线路参数的初猜。在本文档中，为方便进行演示，以前文Strategy 1中尺寸的两倍作为目标尺寸，以前文的参数作为新线路参数的初猜。

In [12]:
num2 = 2 * num
m2 = NMtable[num2]
print(num2, m2)

8 3


构建大系统的ansatz和哈密顿量。

In [14]:
encoder = Circuit()
for i in range(0, num2, 2):
    encoder += X.on(i)
    encoder += X.on(i + 1)
    encoder += H.on(i)
    encoder += X.on(i + 1, i)

ansatz = Circuit()
count = 1

for j in range(m2):
    for i in range(0, num2 - 1, 2):
        ansatz += N(count, i)
        count += 1
    for i in range(1, num2 - 1, 2):
        ansatz += N(count, i)
        count += 1
    for i in range(0, num2 // 2):
        ansatz += PhaseShift('p%d' % count).on(i)
        ansatz += PhaseShift({'p%d' % count : -1}).on(num2 - i - 1)
        count += 1

ansatz = encoder + ansatz
ansatz.summary()

J = 1
ham = QubitOperator('')
for i in range(num2 - 1):
    ham += QubitOperator('X%d X%d' % (i, i + 1), J)
    ham += QubitOperator('Y%d Y%d' % (i, i + 1), J)
    ham += QubitOperator('Z%d Z%d' % (i, i + 1), J)
ham = Hamiltonian(ham - QubitOperator(''))

print(ham)
ansatz

===========================Circuit Summary===========================
|Total number of gates  : 271.                                      |
|Parameter gates        : 87.                                       |
|with 33 parameters are : p1, p2, p3, p4, p5, p6, p7, p8, p9, p10...|
|Number qubit of circuit: 8                                         |
1 [X0 X1] +
1 [Y0 Y1] +
1 [Z0 Z1] +
1 [X1 X2] +
1 [Y1 Y2] +
1 [Z1 Z2] +
1 [X2 X3] +
1 [Y2 Y3] +
1 [Z2 Z3] +
1 [X3 X4] +
1 [Y3 Y4] +
1 [Z3 Z4] +
1 [X4 X5] +
1 [Y4 Y5] +
1 [Z4 Z5] +
1 [X5 X6] +
1 [Y5 Y6] +
1 [Z5 Z6] +
1 [X6 X7] +
1 [Y6 Y7] +
1 [Z6 Z7] 


q0: ──X────H────●───────────────X────RZ(2*p1)─────RZ(-π/2)────●────────────────────────────X────RZ(-π/2)─────PS(p8)─────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────────RZ(2*p12)─────RZ(-π/2)────────●────────────────────────────────────X───────RZ(-π/2)─────PS(p19)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────────RZ(2*p23)─────RZ(-π/2)────────●─────────────────────────────────────X─────────RZ(-π/2)──────PS(p30)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                │               │                             │                            │                                                                                                                                    │                                       │                                    │                                                                                                                                                                      │                                       │                                     │
q1: ──X─────────X────RZ(π/2)────●────RY(π/2)─────RY(-2*p1)────X────RY(2*p1)────RY(-π/2)────●───────────────────────────X────RZ(2*p5)─────RZ(-π/2)────●────────────────────────────X────RZ(-π/2)─────PS(p9)──────RZ(π/2)─────────●──────────RY(π/2)─────RY(-2*p12)───────X────────RY(2*p12)────RY(-π/2)───────●────────────────────────────────────X─────────RZ(2*p16)─────RZ(-π/2)────────●────────────────────────────────────X─────────RZ(-π/2)─────PS(p20)───────RZ(π/2)─────────●──────────RY(π/2)─────RY(-2*p23)───────X────────RY(2*p23)────RY(-π/2)────────●───────────────────────────────────────X─────────RZ(2*p27)─────RZ(-π/2)────────●────────────────────────────────────X───────RZ(-π/2)────PS(p31)──
                                                                                                                       │                             │                            │                                                                                                                                                               │                                       │                                    │                                                                                                                                                                          │                                       │                                    │
q2: ──X────H────●───────────────X────RZ(2*p2)─────RZ(-π/2)────●────────────────────────────X────RZ(-π/2)────RZ(π/2)────●────RY(π/2)─────RY(-2*p5)────X────RY(2*p5)────RY(-π/2)────●────PS(p10)──────────────────────────────────X─────────RZ(2*p13)─────RZ(-π/2)────────●────────────────────────────────────X───────RZ(-π/2)─────RZ(π/2)─────────●──────────RY(π/2)─────RY(-2*p16)───────X────────RY(2*p16)────RY(-π/2)───────●─────────PS(p21)────────────────────────────────────X─────────RZ(2*p24)─────RZ(-π/2)────────●─────────────────────────────────────X─────────RZ(-π/2)──────RZ(π/2)─────────●──────────RY(π/2)─────RY(-2*p27)───────X────────RY(2*p27)────RY(-π/2)───────●───────PS(p32)──────────────
                │               │                             │                            │                                                                                                                                    │                                       │                                    │                                                                                                                                                                      │                                       │                                     │
q3: ──X─────────X────RZ(π/2)────●────RY(π/2)─────RY(-2*p2)────X────RY(2*p2)────RY(-π/2)────●───────────────────────────X────RZ(2*p6)─────RZ(-π/2)────●──────────

调取Strategy 1得到的参数，作为新的线路参数的初猜。论文中指出，如果新的线路的层数大于先前的线路，那么旧参数只应用于原先的层数，多出的层数仍进行随机初始化。此外，对于连接上下两半的一个N门的参数也进行随机初始化。

In [13]:
pre = pqcnet.weight.asnumpy()
half = num // 2
L1 = half * 3 - 1
L2 = num * 3 - 1
print(L1, L2)
val = np.random.randn(L2 * m2)
for j in range(m):
    for i in range(half):
        val[i + j * L2] = pre[i + j * L1]
        val[i + j * L2 + half] = pre[i + j * L1]
for j in range(m):
    for i in range(half - 1):
        val[i + j * L2 + num] = pre[i + j * L1 + half]
        val[i + j * L2 + num + half] = pre[i + j * L1 + half]
for j in range(m):
    for i in range(half):
        val[i + j * L2 + num2 - 1] = pre[i + j * L1 + num - 1]
        val[num - i - 2 + j * L2 + num2] = -pre[i + j * L1 + num - 1]

print('old parameters:')
for i in range(L1 * m):
    print(f'p{i + 1}', pre[i])
print('new parameters:')
for i in range(L2 * m2):
    print(f'p{i + 1}', val[i])

5 11
old parameters:
p1 0.4028975
p2 1.7767195
p3 0.78539985
p4 -0.44342345
p5 0.44359204
p6 -0.23663777
p7 0.5444191
p8 -0.54652977
p9 -1.57108
p10 -1.5708411
new parameters:
p1 0.40289750695228577
p2 1.7767194509506226
p3 0.40289750695228577
p4 1.7767194509506226
p5 0.785399854183197
p6 -0.7981621709703667
p7 0.785399854183197
p8 -0.44342344999313354
p9 0.44359204173088074
p10 -0.44359204173088074
p11 0.44342344999313354
p12 -0.23663777112960815
p13 0.5444191098213196
p14 -0.23663777112960815
p15 0.5444191098213196
p16 -0.5465297698974609
p17 -0.34117606704058145
p18 -0.5465297698974609
p19 -1.571079969406128
p20 -1.5708410739898682
p21 1.5708410739898682
p22 1.571079969406128
p23 0.17239491160199313
p24 -0.914783228142229
p25 -0.590292046100649
p26 0.5591994325996155
p27 1.4652410773726405
p28 -0.40170510817277555
p29 1.2007430289046384
p30 -1.0807505299102804
p31 -0.08635083542922928
p32 0.6355362236237798
p33 -0.20765632405686124


在新的PQC上使用Adam算法进行梯度优化。

In [14]:
sim = Simulator('mqvector', num2)
pqc = sim.get_expectation_with_grad(ham, ansatz)
pqcnet = MQAnsatzOnlyLayer(pqc)
pqcnet.weight = Parameter(ms.Tensor(val, pqcnet.weight.dtype))

initial_energy = pqcnet()
print("Initial energy: %20.16f" % (initial_energy.asnumpy()))

optimizer = ms.nn.Adam(pqcnet.trainable_params(), learning_rate = 0.01, beta1 = 0.9, beta2 = 0.999, eps = 1e-8)
train_pqcnet = ms.nn.TrainOneStepCell(pqcnet, optimizer)

eps = 1.e-8
print("eps: ", eps)
energy_diff = eps * 1000
energy_last = initial_energy.asnumpy() + energy_diff
iter_idx = 0
while (abs(energy_diff) > eps):
    energy_i = train_pqcnet().asnumpy()
    if iter_idx % 100 == 0:
        print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
    energy_diff = energy_last - energy_i
    energy_last = energy_i
    iter_idx += 1

print("Optimization completed at step %3d" % (iter_idx - 1))
print("Optimized energy: %20.16f" % (energy_i))
print("Optimized amplitudes: \n", pqcnet.weight.asnumpy())

Initial energy:  -0.7443327307701111
eps:  1e-08
Step   0 energy  -0.7443327307701111
Step 100 energy -12.1580400466918945
Step 200 energy -12.3319196701049805
Step 300 energy -12.5329952239990234
Step 400 energy -12.8005189895629883
Step 500 energy -13.1239566802978516
Step 600 energy -13.2102670669555664
Step 700 energy -13.2432384490966797
Step 800 energy -13.2575006484985352
Step 900 energy -13.2690734863281250
Step 1000 energy -13.2739782333374023
Step 1100 energy -13.2763023376464844
Step 1200 energy -13.2777204513549805
Step 1300 energy -13.2787160873413086
Step 1400 energy -13.2794694900512695
Step 1500 energy -13.2800588607788086
Step 1600 energy -13.2805252075195312
Step 1700 energy -13.2808961868286133
Step 1800 energy -13.2811956405639648
Step 1900 energy -13.2814435958862305
Step 2000 energy -13.2816562652587891
Step 2100 energy -13.2818431854248047
Step 2200 energy -13.2820081710815430
Step 2300 energy -13.2821502685546875
Step 2400 energy -13.2822694778442383
Optimizatio

Strategy 2完整源代码见src/qubit recursive.py，相较本文档，源代码从目标的$N$出发，尽可能从$N$中除掉2的幂，再往上倍增至$N$。

值得一提的是，论文中并没有明确给出子系统的收敛的判据。论文前文在随机初始化的段落给出“对约100个不同的随机初始化参数，只有它们全部都能在50L次循环后达到保真度阈值，才判定算法成功”，这个判据显然不能应用在这个场景里，因为实际VQE的过程中目标态是未知的，无法计算保真度，只能比较更新步骤之间的能量差。从极端情形考虑，如果收敛判据取得非常严苛（阈值取得非常小），那么大量的循环都浪费在子系统PQC的收敛上，反而达不到加速的效果。

由此考虑，可以将子系统收敛的阈值取得稍大一些，因为通过改善子系统的基态的精度来提高目标系统的精度，这件事情的效率是相对较低的。绝大部分的经典迭代应该用在最终的目标系统上。由于参数的不确定性，无法从理论上给出这一“效率”的大小，从经验上不妨试着取子系统阈值为后一步2倍大系统的10或100倍。

### 改进Strategy 2

论文中指出，使用Strategy 2的动机是：单独考虑子系统的基态(the ground state of the local system)，与大系统的基态的子系统空间(the reduced density matrix of the subsystems from the ground state of the large system)很相似。从物理的角度分析，这个说法肯定是合理的，因为哈密顿量可以改写为$H_N=J (\sum_{i=1}^{N/2-1} \sigma^{i} \cdot \sigma^{i+1}+\sum_{i=N/2}^{N-1} \sigma^{i} \cdot \sigma^{i+1}+\sigma^{N/2-1} \cdot \sigma^{N/2})$.这个式子的前两项分别对应两个子系统，把它们的和看作一个整体$H_{base}$，它的基态就是两个$H_{N/2}$的基态的张量积$|\Psi_{base}\rangle=|\Psi_{N/2}\rangle \otimes |\Psi_{N/2}\rangle$，而真正的哈密顿量$H_N$可以看作是$H_{base}$上的微扰$H_N=H_{base}+\sigma^{N/2-1} \cdot \sigma^{N/2}$，根据微扰论的知识，$N$越大，$|\Psi_{base}\rangle$与$|\Psi_{N}\rangle$的差距就越小。

既然如此，为什么不从物理上认可的$H_{base}$出发作变分呢？想要制备$H_{base}$，就应该让连接上下两半的N门的参数取0，如果新增加了线路层数，那么所有的新添加的参数门的参数也应都取0.论文中将的连接的门和新添加的门的参数随机初始化，是否反而是在南辕北辙？可以做一个简单的验证：从前文中获取的子网络参数pre重新构建线路，Attempt1将连接的门的参数设置为0，Attempt2将连接的门和新添加的门的参数全都设置为0，比较这两者的能量与之前的线路的能量的高低。

In [15]:
pre = pqcnet.weight.asnumpy()
half = num // 2
L1 = half * 3 - 1
L2 = num * 3 - 1

count1 = 0
count2 = 0
numoftests = 1000
for k in range(numoftests):
    val = np.random.randn(L2 * m2)

    for j in range(m):
        for i in range(half):
            val[i + j * L2] = pre[i + j * L1]
            val[i + j * L2 + half] = pre[i + j * L1]
    for j in range(m):
        for i in range(half - 1):
            val[i + j * L2 + num] = pre[i + j * L1 + half]
            val[i + j * L2 + num + half] = pre[i + j * L1 + half]
    for j in range(m):
        for i in range(half):
            val[i + j * L2 + num2 - 1] = pre[i + j * L1 + num - 1]
            val[num - i - 2 + j * L2 + num2] = -pre[i + j * L1 + num - 1]

    sim = Simulator('mqvector', num2)
    pqc = sim.get_expectation_with_grad(ham, ansatz)
    pqcnet = MQAnsatzOnlyLayer(pqc)
    pqcnet.weight = Parameter(ms.Tensor(val, pqcnet.weight.dtype))

    initial_energy1 = pqcnet()
    #print("Paper Initial energy: %20.16f" % (initial_energy.asnumpy()))

    for j in range(m):
        val[j * L2 + num + half - 1] = 0
    pqcnet.weight = Parameter(ms.Tensor(val, pqcnet.weight.dtype))
    initial_energy2 = pqcnet()
    #print("Attempt 1 Initial energy: %20.16f" % (initial_energy.asnumpy()))

    val = np.zeros(L2 * m2)
    for j in range(m):
        for i in range(half):
            val[i + j * L2] = pre[i + j * L1]
            val[i + j * L2 + half] = pre[i + j * L1]
    for j in range(m):
        for i in range(half - 1):
            val[i + j * L2 + num] = pre[i + j * L1 + half]
            val[i + j * L2 + num + half] = pre[i + j * L1 + half]
    for j in range(m):
        for i in range(half):
            val[i + j * L2 + num2 - 1] = pre[i + j * L1 + num - 1]
            val[num - i - 2 + j * L2 + num2] = -pre[i + j * L1 + num - 1]
    pqcnet.weight = Parameter(ms.Tensor(val, pqcnet.weight.dtype))
    initial_energy3 = pqcnet()
    #print("Attempt 2 Initial energy: %20.16f" % (initial_energy.asnumpy()))
    if initial_energy2 < initial_energy1:
        count1 +=1
    if initial_energy3 < initial_energy1:
        count2 +=1
print('Attempt 1 better: %.1f' % (100.0*count1/numoftests) + '%')
print('Attempt 2 better: %.1f' % (100.0*count2/numoftests) + '%')

Attempt 1 better: 57.4%
Attempt 2 better: 74.5%


可以看到大多数情况下，随机初始化将量子态偏离$|\Psi_{base}\rangle$，其得到的结果反而不如$|\Psi_{base}\rangle$，因此不如就将$|\Psi_{base}\rangle$作为大系统变分的起点。整体线路的随机性由最小的子系统的参数的随机初始化实现。

在src/qubit recursive.py中，解除第165、166行的注释即可实现Attempt1，解除第167行的注释即可实现Attempt2.

### Strategy 3: layer recursive.

这个策略先从单层的PQC出发，达到收敛之后，增加一层PQC的层数（使用前一层的参数作为初猜），达到收敛之后再增加一层，以此类推直至总层数达到$M_{VQE}$.

论文中没有给出使用这一策略的理论依据。个人认为这一策略的可以从几何上理解，在经过VQE的变分优化后，如果恰好初态、末态和目标态三者共面，那么下一步的参数的最优取值一定是前一步的复制，不共面时也是不错的贪心策略。这与Grover算法的思路类似。

In [16]:
num = 4
m = NMtable[num]
assert num % 2 == 0
#print(num, m)


encoder = Circuit()
for i in range(0, num, 2):
    encoder += X.on(i)
    encoder += X.on(i + 1)
    encoder += H.on(i)
    encoder += X.on(i + 1, i)
    
#print(encoder)
#encoder.summary()


J = 1
ham = QubitOperator('')
for i in range(num - 1):
    ham += QubitOperator('X%d X%d' % (i, i + 1), J)
    ham += QubitOperator('Y%d Y%d' % (i, i + 1), J)
    ham += QubitOperator('Z%d Z%d' % (i, i + 1), J)
ham -= QubitOperator('')

#print(ham)


ansatz = encoder
count = 1
sim = Simulator('mqvector', num)
val = []
L = num // 2 * 3 - 1

for j in range(m):
    print(f'Current number of layers: {j + 1}')
    # print(val)
    for i in range(0, num - 1, 2):
        ansatz += N(count, i)
        count += 1
    for i in range(1, num - 1, 2):
        ansatz += N(count, i)
        count += 1
    for i in range(0, num // 2):
        ansatz += PhaseShift('p%d' % count).on(i)
        ansatz += PhaseShift({'p%d' % count : -1}).on(num - i - 1)
        count += 1

    # print(ansatz)
    # ansatz.summary()

    if j == 0:
        val = np.random.randn(count - 1)
    else:
        val = np.concatenate((val, val[-L:]))
    # print(val)

    
    pqc = sim.get_expectation_with_grad(Hamiltonian(ham), ansatz)
    pqcnet = MQAnsatzOnlyLayer(pqc)
    pqcnet.weight = Parameter(ms.Tensor(val, pqcnet.weight.dtype))

    initial_energy = pqcnet()
    print("Initial energy: %20.16f" % (initial_energy.asnumpy()))


    optimizer = ms.nn.Adam(pqcnet.trainable_params(), learning_rate = 0.01, beta1 = 0.9, beta2 = 0.999, eps = 1e-8)
    train_pqcnet = ms.nn.TrainOneStepCell(pqcnet, optimizer)

    eps = 1.e-8
    print("eps: ", eps)
    energy_diff = eps * 1000
    energy_last = initial_energy.asnumpy() + energy_diff
    iter_idx = 0
    while (abs(energy_diff) > eps):
        energy_i = train_pqcnet().asnumpy()
        if iter_idx % 100 == 0:
            print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
        energy_diff = energy_last - energy_i
        energy_last = energy_i
        iter_idx += 1

    print("Optimization completed at step %3d" % (iter_idx - 1))
    print("Optimized energy: %20.16f" % (energy_i))
    print("Optimized amplitudes: \n", pqcnet.weight.asnumpy())

Current number of layers: 1
Initial energy:   0.0123526062816381
eps:  1e-08
Step   0 energy   0.0123526062816381
Optimization completed at step  94
Optimized energy:   0.0000000000238039
Optimized amplitudes: 
 [-1.8926561   0.168481   -0.78533477 -0.46012077 -1.4066193 ]
Current number of layers: 2
Initial energy:  -2.3261110782623291
eps:  1e-08
Step   0 energy  -2.3261110782623291
Step 100 energy  -5.9998421669006348
Optimization completed at step 124
Optimized energy:  -5.9999876022338867
Optimized amplitudes: 
 [-1.8926561   0.168481   -0.78522205 -0.32984713 -0.8735825  -1.8141392
  0.24335901 -0.78527933 -0.32926998 -1.5350333 ]


Strategy 3完整源代码见src/layer recursive.py

## 项目总结

### 复现结果
实现了论文Sec.5中的3种不同策略的VQE线路，3种都可以复现N=4、8、10时海森堡模型的基态能量。
### 创新点
改进了Strategy 2中的参数初猜设定
### 未来可继续研究的方向
1. 论文绘制了许多与保真度相关的图表，由于mindquantum没有内置的矩阵严格对角化的功能，这些图表若要在mindquantum中重现，需要从外部导入严格对角化的结果，与ansatz.get_qs(pr = val)的结果作内积，这里由于位序的定义问题很容易引发错误。可以考虑内置矩阵严格对角化。
2. 由于密度矩阵模块尚不健全，无法模拟双量子比特门的失相，可以考虑引入密度矩阵，Kraus算符等模块。
3. 文中固定了使用Adam算法，取了一组特定的学习率等参数，可以探索不同的经典优化算法，不同参数之间的性能差异。
4. 可以探索绝热演化转VQE这一线路设计范式在其他模型中的应用。